<a href="https://colab.research.google.com/github/lokeshdarla/cat-vs-dog/blob/main/colab-notebooks/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CAT vs DOG classifier

data link [https://www.kaggle.com/datasets/salader/dogs-vs-cats](https://www.kaggle.com/datasets/salader/dogs-vs-cats)

In [3]:
! mkdir ~p ~/.kaggle
! cp kaggle.json ~/.kaggle/

mkdir: cannot create directory ‘~p’: File exists


In [4]:
!kaggle datasets download -d salader/dogs-vs-cats

Dataset URL: https://www.kaggle.com/datasets/salader/dogs-vs-cats
License(s): unknown
100% 1.06G/1.06G [00:12<00:00, 98.0MB/s]
100% 1.06G/1.06G [00:12<00:00, 92.8MB/s]


In [5]:
import zipfile
zip_ref = zipfile.ZipFile('/content/dogs-vs-cats.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [1]:
print("Hello World")

Hello World
